In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yaml
#from yaml import CLoader

In [ ]:
%%time

with open('./ncbi_dataset/data/data_report.yaml') as f:
    genomes = yaml.load(f, Loader=yaml.CLoader)

In [ ]:
pickle.dump(genomes, open('../data/2020_08_18_genomes.pickle', 'w'))

In [ ]:
len(genomes['genomes'])

In [ ]:
dict.

In [ ]:
def drop_dict(d, key):
    """Drops (or pops) a key from a dictionary and (unlike .pop) return the new dictionary"""
    try: 
        d.pop(key)
    except KeyError: 
        pass #if key is missing, just proceed
    return d

In [ ]:
[genome['virus']['sciName'] for genome in genomes['genomes']]

In [ ]:
genomes_noanno = [drop_dict(genome, 'annotation') for genome in genomes['genomes']]

In [ ]:
genomes_noanno

In [ ]:
genomes_df = pd.DataFrame(genomes_noanno)

genomes_df['sciName'] = genomes_df['virus'].apply(lambda x: x['sciName'])

In [ ]:
genomes_df.groupby('sciName').count().sort_values('accession', ascending=False)

In [ ]:
genomes_df[['length']].plot.hist(bins=30)

In [ ]:
genomes_df[['length']][genomes_df['length'] > 20000].plot.hist(bins=30)

In [ ]:
genomes_df.groupby('sciName').count().plot(y='accession')

In [ ]:
genomes_df.groupby('sciName').count().sort_values('accession', ascending=False).head(10).plot(y='accession', kind='barh')

In [ ]:


genomes_df['location'].dropna().apply(lambda x: x['geographicLocation'])

In [ ]:
import geopandas

In [ ]:
from geopy.geocoders import GeocodeFarm

import geopy

In [ ]:
geolocator= GeocodeFarm()

In [ ]:
example = geolocator.geocode('China: Hong Kong, Prince of Wales Hospital')

example

In [ ]:
example.latitude

In [ ]:
genomes_df['location'].dropna().apply(lambda x: x['geographicLocation']).unique()

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="coronavirus_genome_mapper")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)





In [ ]:
#cache = dict()
#counter = 0
#cached_counter = 0

def cached_geocode(location):
    global counter, cached_counter, cache
    if (counter % 100) == 0:
        print(f"geocoded {counter} locations ({cached_counter/max(1, counter)}% cached)")
    counter += 1
    if location in cache:
        cached_counter += 1
        return cache[location]
    else:
        cache[location] = geocode(location)
        return cache[location]

In [ ]:
cached_geocode("chicago")

In [ ]:
cached_counter

In [ ]:
import functools

In [ ]:
genomes_df_top = genomes_df.head(10)

In [ ]:
genomes_df['point'] = genomes_df['location'].apply(lambda loc: cached_geocode(loc['geographicLocation'] if type(loc) != float else None))

In [ ]:
genomes_df

In [ ]:
hasattr

In [ ]:
def get_point(loc):
    try:
        return (loc.latitude, loc.longitude, 0)
    except AttributeError:
        return None

In [ ]:
 genomes_df['point'].apply(get_point)

In [ ]:
genomes_df['point2'] = genomes_df['point'].apply(get_point)

genomes_coords = pd.DataFrame(genomes_df['point2'].to_list())

genomes_coords.columns = ["lat", 'lon', 'elevation']

genomes_df2 = pd.concat([genomes_df, genomes_coords], axis=1)

In [ ]:
genomes_df2.to_csv("../output/2020_08_19_Genomes Geocoded.csv")

In [ ]:
def lon_to_web_mercator(lon):
    k = 6378137
    return lon * (k * np.pi / 180.0)


def lat_to_web_mercator(lat):
    k = 6378137
    return np.log(np.tan((90 + lat) * np.pi / 360.0)) * k


def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["lon"] = df[lon] * (k * np.pi / 180.0)
    df["lat"] = np.log(np.tan((90 + df[lat]) * np.pi / 360.0)) * k
    return df


In [ ]:
locations = pd.DataFrame(dict(lat=[ 30.29,  30.20,  30.29],
              lon=[-97.70, -97.74, -97.78]))

In [ ]:
#with pd.option_context('display.max_rows', 10**6):
#pd.set_option('display.max_rows', None)
top_ten = genomes_df2.groupby('sciName').count().sort_values('accession').tail(10)

top_ten = top_ten.index.to_list()


In [ ]:
top_ten

In [ ]:
from bokeh.palettes import Set3  


Set3[]

In [ ]:
pd.set_option('display.max_rows', 100)


In [ ]:
from bokeh.palettes import Spectral11
factor_cmap('type', palette=Spectral11, factors=top_ten)

In [ ]:
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral11

#output_file("tile.html")
output_notebook()

tile_provider = get_provider(CARTODBPOSITRON)

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)


#source = ColumnDataSource(
#    data=wgs84_to_web_mercator(locations))

p.circle(x="lon", y="lat", size=15, 
         #fill_color="blue", 
         fill_color=factor_cmap('sciName', palette=Spectral11, factors=top_ten),
         fill_alpha=0.1, source=wgs84_to_web_mercator(genomes_df2.loc[:,["lat", "lon", 'sciName']]))


#output_notebook(p)
show(p)


In [ ]:
genomes_df2.head()

In [ ]:
genomes_df2.loc[:,["lat", "lon"]]

In [ ]:
eval("""{'taxId': 9606, 'sciName': 'Homo sapiens', 'lineage': [{'name': 'cellular organisms', 'taxId': 131567}, {'name': 'Eukaryota (eucaryotes)', 'taxId': 2759}, {'name': 'Opisthokonta', 'taxId': 33154}, {'name': 'Metazoa (metazoans)', 'taxId': 33208}, {'name': 'Eumetazoa', 'taxId': 6072}, {'name': 'Bilateria', 'taxId': 33213}, {'name': 'Deuterostomia (deuterostomes)', 'taxId': 33511}, {'name': 'Chordata (chordates)', 'taxId': 7711}, {'name': 'Craniata', 'taxId': 89593}, {'name': 'Vertebrata (vertebrates)', 'taxId': 7742}, {'name': 'Gnathostomata (jawed vertebrates)', 'taxId': 7776}, {'name': 'Teleostomi', 'taxId': 117570}, {'name': 'Euteleostomi (bony vertebrates)', 'taxId': 117571}, {'name': 'Sarcopterygii', 'taxId': 8287}, {'name': 'Dipnotetrapodomorpha', 'taxId': 1338369}, {'name': 'Tetrapoda (tetrapods)', 'taxId': 32523}, {'name': 'Amniota (amniotes)', 'taxId': 32524}, {'name': 'Mammalia (mammals)', 'taxId': 40674}, {'name': 'Theria', 'taxId': 32525}, {'name': 'Eutheria (placentals)', 'taxId': 9347}, {'name': 'Boreoeutheria', 'taxId': 1437010}, {'name': 'Euarchontoglires', 'taxId': 314146}, {'name': 'Primates', 'taxId': 9443}, {'name': 'Haplorrhini', 'taxId': 376913}, {'name': 'Simiiformes', 'taxId': 314293}, {'name': 'Catarrhini', 'taxId': 9526}, {'name': 'Hominoidea (apes)', 'taxId': 314295}, {'name': 'Hominidae (great apes)', 'taxId': 9604}, {'name': 'Homininae', 'taxId': 207598}, {'name': 'Homo (humans)', 'taxId': 9605}, {'name': 'Homo sapiens (human)', 'taxId': 9606}]}""")